In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from boundary import *
import rootfinding as rf
custom_rf = {'tol': 2E-12, 'maxiter': 50,
             'rootfind_open': rf.newton, 'rootfind_bracketing': rf.bisect}

In [2]:
def billiard_propagator_gen(r, v, bdy):
    s = bdy.linear_intersect_cart(r, v)
    if np.any(np.sign(bdy.coords_cart(s) - r) != np.sign(v)):
        # Then we found the intercept in the wrong direction! Try again.
        try:
            s = bdy.linear_intersect_param(s, v)  # Excludes passed root s
        except RuntimeError as e:
            print e
    while True:
        tangent = bdy.tangent_cart(s)
        v_tan = np.dot(v, tangent)
        yield s, v_tan
        v = 2 * v_tan * tangent - v
        try:
            s = bdy.linear_intersect_param(s, v)
        except RuntimeError as e:
            print e

def init_state(r, v, boundary):
    state = {'s_list': [],
             'v_tan_list': [],
             'trajectory': np.c_[r],
             'propagator': billiard_propagator_gen(r, v, boundary)}
    return state


def propagate(state, bounces):
    new_s, new_v_tan = zip(*(state['propagator'].next()
                             for _ in xrange(bounces)))
    state['s_list'].extend(new_s)
    state['v_tan_list'].extend(new_v_tan)
    state['trajectory'] = np.c_[state['trajectory'],
                                bdy.coords_cart(np.array(new_s))]

In [3]:
%matplotlib auto
r0 = np.array([0.6, 0.2])
theta = 0.1
v0 = np.array([np.cos(theta), np.sin(theta)])
#bdy = UnitCircleBoundary(**custom_rf)
bdy = BeanBoundary(0.16, 0.1, 2.0, 1.0, **custom_rf)

default_bounces = 20
max_bounces = 500
default_deltay = 0
max_deltay = 10

fig, axlist = plt.subplots(1, 2)
bdyline = bdy.coords_cart(np.arange(0, 2*np.pi, 0.01))
axlist[0].plot(bdyline[0], bdyline[1])
# Plot trajectory line object; I'll send data to it later 
traj_line, = axlist[0].plot([0, 0], [0, 0], 'o-', label='Trajectory')
poinc_ax = axlist[1]
plt.axis('equal')

# Make Sliders (with their own axis artists) for interactive control.
axcolor = 'lightgoldenrodyellow'
axbounces = plt.axes([0.2, 0.03, 0.65, 0.02])#, facecolor=axcolor)
sbounces = Slider(axbounces, 'Number of bounces',
                  1, max_bounces, valinit=default_bounces)
axdeltay = plt.axes([0.2, 0.005, 0.65, 0.02])#, facecolor=axcolor)
sdeltay = Slider(axdeltay, r'$\Delta y_0$ / $(y_0 * 10^{-8})$',
                 -max_deltay, max_deltay, valinit=default_deltay)


def propagate_plots(state, n, traj_line, poinc_ax):
    missing = n+1 - state['trajectory'].shape[1]
    poinc_ax.clear()
    if missing <= 0:
        traj_line.set_xdata(state['trajectory'][0, 0:n+1])
        traj_line.set_ydata(state['trajectory'][1, 0:n+1])
        poinc_ax.scatter(state['s_list'][0:n+1],
                         state['v_tan_list'][0:n+1])
    else:
        propagate(state, missing)    
        traj_line.set_xdata(state['trajectory'][0])
        traj_line.set_ydata(state['trajectory'][1])
        poinc_ax.scatter(state['s_list'],
                         state['v_tan_list'])
    fig.canvas.draw()


def reset_plots(r, v, boundary, n, traj_line, poinc_ax):
    state = init_state(r, v, boundary)
    propagate_plots(state, n, traj_line, poinc_ax)
    return state

# Initialize plotline data for default Slider settings.
state = reset_plots(r0 * np.array([1, 1 + sdeltay.val*5e-7]), v0, bdy,
                    int(sbounces.val), traj_line, poinc_ax)
# Configure our Sliders to update the plotlines.
# The Sliders will pass their current value to the on_changed event callback
sbounces.on_changed(lambda val: propagate_plots(state, int(val),
                                                traj_line, poinc_ax))
sdeltay.on_changed(lambda val: reset_plots(r0 * np.array([1, 1 + val*1e-8]),
                                           v0, bdy, int(sbounces.val),
                                           traj_line, poinc_ax))

# plt.show()

Using matplotlib backend: TkAgg


TypeError: can't multiply sequence by non-int of type 'float'

In [ ]:
def angular_distance(s1, s2):
    """Return the signed distance in range [-pi, pi] from s1 to s2,
        where s1 and s2 are angles between 0 and 2pi.
    """
    d = s2 - s1
    # want elementwise: d if |d| <= pi else sgn(d)*(|d| - 2pi) 
    absd = np.abs(d)
    sign = np.sign(d)
    boo = absd > np.pi  # boolean array for indexing
    d[boo] = -sign[boo]*(absd[boo] - 2*np.pi)  
    return d

In [ ]:
def angular_distance(s1, s2):
    """Return the signed distance in range [-pi, pi] from s1 to s2,
        where s1 and s2 are angles between 0 and 2pi.
    """
    d = s2 - s1
    # want elementwise: d if |d| <= pi else sgn(d)*(|d| - 2pi) 
    absd = np.abs(d)
    sign = np.sign(d)
    boo = absd > np.pi  # boolean array for indexing
    d[boo] = -sign[boo]*(absd[boo] - 2*np.pi)  
    return d

bounces = 50

def get_pert_deltas(dys, ref_bdy, s_arr_0, r0, v0):
    deltas_list = []
    for dy in dys:
        state = init_state(r0 * np.array([1, 1 + dy]), v0, ref_bdy)
        propagate(state, bounces)
        deltas_list += [angular_distance(np.array(state['s_list']), s_arr_0)]
    return deltas_list

def get_tol_deltas(tols, ref_bdy, s_arr_0, r0, v0):
    deltas_list = []
    ref_tol = ref_bdy.tol
    for tol in tols:
        ref_bdy.tol = tol
        state = init_state(r0, v0, ref_bdy)
        propagate(state, bounces)
        deltas_list += [angular_distance(np.array(state['s_list']), s_arr_0)]
    ref_bdy.tol = ref_tol
    return deltas_list

def plot_divergence(ref_bdy, r0, v0, lines):
    reference = init_state(r0, v0, ref_bdy)
    propagate(reference, bounces)
    s_arr_0 = np.array(reference['s_list'])

    dys = np.linspace(0, 5e-9, num=lines+1)[1::]
    deltas_pert_list = get_pert_deltas(dys, ref_bdy, s_arr_0, r0, v0)

    tols = r0[1]*dys
    deltas_tol_list = get_tol_deltas(tols, ref_bdy, s_arr_0, r0, v0)

    fig, axlist = plt.subplots(2, 1, sharex=True)
    for deltas_pert, dy, deltas_tol, tol in zip(deltas_pert_list, dys,
                                                deltas_tol_list, tols):
        axlist[0].plot(deltas_pert, label='$\Delta y_0$ = {0:.2e}'.format(r0[1] * dy))
        axlist[1].plot(deltas_tol, label='Tolerance = {0:.2e}'.format(tol))
    axlist[0].legend(loc='lower left')
    axlist[1].legend(loc='lower left')

In [ ]:
# r0 = np.array([0.6, 0.2])
# theta = 0.1
r0 = np.array([0.1, 0])
theta = 1.5
v0 = np.array([np.cos(theta), np.sin(theta)])
custom_rf['param_rootfind': 'open']
for ref_bdy in (UnitCircleBoundary(**custom_rf),
                BeanBoundary(0.16, 0.1, 2.0, 1.0, **custom_rf)):
    plot_divergence(ref_bdy, r0, v0, 10)